In [15]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import webbrowser
import requests
import random
import string
from selenium import webdriver
import time
import pandas as pd

In [ ]:
profile_directory = "C:/Users/Administrator/AppData/Roaming/Mozilla/Firefox/Profiles/t9g6gqph.default-1572349721351"
# 加载配置
profile = webdriver.FirefoxProfile(profile_directory)
# 启动浏览器配置
driver = webdriver.Firefox(profile)

driver.get("https://www.51job.com/")
driver.find_element_by_id("kwdselectid").send_keys(u"计算机视觉")
driver.find_element_by_xpath(u".//*[@class='content']/div/div/div/button").click()# 模仿人的点击命令，由浏览器发出post
soup = BeautifulSoup(driver.page_source, 'lxml') # 将点击后返回的页面做成soup
first_page_content = soup.find("div",{"id":"resultList"})
# 获取一共多少个结果
number_con = first_page_content.find("div",{"class":"dw_tlc"})
number = number_con.find_all("div",{"class":"rt"})
print(number[0].get_text())
# 开始从页面爬取数据
want_number = input("想要爬取的数量")
want_number = int(want_number)
data = []
while want_number >= 0:
    page_cont = soup.find("div",{"id":"resultList"})
    page_list = page_cont.find_all("div",{"class":"el"})
    for i in page_list:
        if i == page_list[0]:
            continue
        # 从每一行职位中提取信息
        name = i.find("p").get_text() # 职位名
        isschool = i.find("p").find("img",{"alt":"校招"})
        ispractice = i.find("p").find("img",{"alt":"实习"})
        company = i.find("span",{"class":"t2"}).get_text()
        location = i.find("span",{"class":"t3"}).get_text()
        pay = i.find("span",{"class":"t4"}).get_text()
        date = i.find("span",{"class":"t5"}).get_text()
        # 数据清洗
        while True:
            """将名字中的换行符和空格全部去掉"""
            name = name.replace(" ","").replace("\n","")
            if " " and "\n" not in name:
                break
        if isschool:
            isschool = 1
        else:
            isschool = 0
        if ispractice:
            ispractice = 1
        else:
            ispractice = 0
        one_data = [name,pay,location,isschool,ispractice,date]
        print(one_data)
        data.append(one_data)
        want_number -= 1
        # 如果在这一页就全部遍历完了的话就结束
        if want_number <= 0 :
            break
    if want_number <= 0:
        break
    next_page_button = driver.find_element_by_link_text('下一页').click()
    soup = BeautifulSoup(driver.page_source, 'lxml')        
df = pd.DataFrame(data,columns=['职位名','薪资','位置','是否校招','是否实习','日期'],dtype=float)
driver.quit()


    共2441条职位



In [17]:
df.head()

,职位名,薪资,位置,是否校招,是否实习,日期
0,计算机视觉实习生,,成都,1.0,1.0,11-01
1,相机与计算机视觉算法工程师,,南京,1.0,0.0,11-01
2,算法研究工程师-机器学习/计算机视觉/AI软件,,北京,1.0,0.0,11-01
3,31329-计算机视觉专家研究员（深圳）,,深圳,0.0,0.0,11-01
4,计算机视觉研发工程师,1.5-2万/月,广州-天河区,0.0,0.0,11-01
